In [2]:

#from common.download_utils import download_week1_resources

#download_week1_resources()


In [23]:
#from grader import Grader
#grader = Grader()
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anmolnarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anmolnarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
from ast import literal_eval
import pandas as pd
import numpy as np
import re


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [26]:
def read_data(filename):
    data = pd.read_csv(filename, sep='\t')
    data['tags'] = data['tags'].apply(literal_eval)
    return data
        

In [27]:
train = read_data('data/train.tsv')
validation = read_data('data/validation.tsv')
test = pd.read_csv('data/test.tsv', sep='\t')
#print(train)

#train.head()

X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values


#tokenizer=nltk.tokenize.TreebankWordTokenizer()
#tokens=tokenizer.tokenize(text);REPLACE_BY_SPACE_RE ='[/(){}\[\]\|@,;?]'


In [28]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = '[^0-9a-z #+_]'
STOPWORDS = set(stopwords.words('english'))
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE," ",text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    
    text = re.sub(BAD_SYMBOLS_RE,"",text)# delete symbols which are in BAD_SYMBOLS_RE from text
    
    tokenizer=nltk.tokenize.TreebankWordTokenizer()
    text=tokenizer.tokenize(text);
    
   # text = word_tokenize(text) 
     # delete stopwords from text
    
    filtered_sentence ="" 
    for w in text: 
        if w not in STOPWORDS: 
            filtered_sentence += w+" "

    return filtered_sentence

In [52]:
'''
#print(X_train[0])
X_train_filtered=[]
for train_data in X_train:
    X_train_filtered.append(text_prepare(train_data))

X_train=X_train_filtered
#print(X_train_filtered[0])
'''


In [53]:
'''
X_val_filtered=[]
for val_data in X_val:
    X_val_filtered.append(text_prepare(val_data))
X_val=X_val_filtered
'''

In [19]:
prepared_questions = []
for line in open('data/text_prepare_tests.tsv', encoding='utf-8'):
    line = text_prepare(line.strip())
    prepared_questions.append(line)
text_prepare_results = '\n'.join(prepared_questions)

In [71]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [22]:
#X_train[0]

'draw stacked dotplot r '

In [55]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}

for index in  range(len(X_train)):
    words=X_train[index].split(' ')
    for word in words:
        if word in words_counts:
            words_counts[word]+=1
        else:
            words_counts[word]=1
#print(words_counts)






In [56]:
for tags in y_train:
    for tag in tags:
        if tag in tags_counts:
            tags_counts[tag] += 1
        else:
            tags_counts[tag] = 1

#print(tags_counts)

In [57]:
print(len(tags_counts))
len(words_counts)
#print(test_text_prepare())
#print(X_train_tokenized[1])

100


31393

In [77]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:10000]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:5000]
#print(most_common_tags)


In [78]:
def words_indexs():
    key_indexes={}
    index=0
    for key,values in most_common_words:
        key_indexes[key]=index
        index+=1
    return key_indexes
#print(key_indexes)

In [79]:
DICT_SIZE = 5000
WORDS_TO_INDEX = words_indexs() ####### YOUR CODE HERE #######
#INDEX_TO_WORDS = ####### YOUR CODE HERE #######
ALL_WORDS = WORDS_TO_INDEX.keys()


def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vecto
from scipy import sparse as sp_sparser which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    words=text.split(' ')
    for word in words:
        if word in words_to_index:
            result_vector[words_to_index[word]]=1

    return result_vector

In [74]:
from scipy import sparse as sp_sparse

In [80]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


In [85]:
#print(X_train_mybag[11])

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf=TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2), token_pattern= '(\S+)')

    X_train=tfidf.fit_transform(X_train);
    X_val=tfidf.fit_transform(X_val);
    X_test=tfidf.fit_transform(X_test);
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    return X_train, X_val, X_test, tfidf.vocabulary_

In [92]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [107]:
if 'c++' in tfidf_vocab:
    print('true')
if 'c#' in tfidf_vocab:
    print('c#')

true
